<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

# SQL window functions
© ExploreAI Academy

In this exercise, we will test our understanding and application of SQL window functions on a sample SQLite database file for a retail company called Northwind by performing complex calculations and analyses like ranking, running totals, and date differences. Ensure that you have downloaded the database file, Northwind.db.

## Learning objectives

By the end of this train, you should:
- Use the RANK() function to assign a ranking number to each row based on the order specified within the window.
- Use aggregate window functions to calculate running totals. 
- Use the LAG() function to help calculate the difference, in days, between consecutive date readings in our dataset.
- Use aggregate window functions to calculate the moving average. 

First, let's load our sample database:

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


In [2]:
# Load the Northwind database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///Northwind.db
    

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Northwind_ERD.png"  style="width:500px";/>
<br>
<br>
    <em>Figure 1: Northwind ERD</em>
</div>

## Exercise

Run the necessary queries that will provide us with the following information. Compare your queries with the solutions at the end of this notebook.

### Exercise 1

Rank all the orders of a specific customer from the most recent to the least recent using window functions. Assume that the customer ID is `'ALFKI'`.

In [9]:
%%sql

SELECT
    orderid,
    orderdate,
    RANK() OVER (
        ORDER BY
            orderdate desc
    )
FROM
    orders
WHERE
    customerid = 'ALFKI'

 * sqlite:///Northwind.db
Done.


OrderID,OrderDate,rank() over (order by orderdate desc)
11011,1998-04-09 00:00:00,1
10952,1998-03-16 00:00:00,2
10835,1998-01-15 00:00:00,3
10702,1997-10-13 00:00:00,4
10692,1997-10-03 00:00:00,5
10643,1997-08-25 00:00:00,6


### Exercise 2

Calculate a running total of the quantity of orders using window functions.

In [22]:
%%sql

SELECT

    orderid,
    quantity,
    sum(quantity) OVER (
        order by orderid
    ) AS running_total_of_the_quantity
FROM
    orderdetails
GROUP BY orderid
LIMIT 10


 * sqlite:///Northwind.db
Done.


OrderID,Quantity,running_total_of_the_quantity
10248,12,12
10249,9,21
10250,10,31
10251,6,37
10252,40,77
10253,20,97
10254,15,112
10255,20,132
10256,15,147
10257,25,172


### Exercise 3


Use window functions to find the difference in successive order dates for each customer. **HINT:** The `TIMESTAMPDIFF()` function in MySQL is not available in SQLite. We can use the `julianday()` function to convert the dates to a floating point number and then calculate the difference.

In [39]:
%%sql

SELECT
    customerid,
    orderdate,
    LAG(orderDate,1) OVER (
        PARTITION BY
            customerid
        ORDER BY
            orderdate
    ) AS orderLAG,
    (julianday(orderDate) - julianday(LAG(orderDate,1) OVER (
        PARTITION BY
            customerid
        ORDER BY
            orderdate
    ))) as diff
FROM
    orders
limit 10

 * sqlite:///Northwind.db
Done.


CustomerID,OrderDate,orderLAG,diff
ALFKI,1997-08-25 00:00:00,None,None
ALFKI,1997-10-03 00:00:00,1997-08-25 00:00:00,39.0
ALFKI,1997-10-13 00:00:00,1997-10-03 00:00:00,10.0
ALFKI,1998-01-15 00:00:00,1997-10-13 00:00:00,94.0
ALFKI,1998-03-16 00:00:00,1998-01-15 00:00:00,60.0
ALFKI,1998-04-09 00:00:00,1998-03-16 00:00:00,24.0
ANATR,1996-09-18 00:00:00,None,None
ANATR,1997-08-08 00:00:00,1996-09-18 00:00:00,324.0
ANATR,1997-11-28 00:00:00,1997-08-08 00:00:00,112.0
ANATR,1998-03-04 00:00:00,1997-11-28 00:00:00,96.0


### Exercise 4

Calculate the moving average of the quantity of the last 3 orders for each product using window functions.

In [54]:
%%sql

SELECT
    orderID,
    productID,
    Quantity,
    AVG(Quantity) OVER (
        PARTITION BY
            productID
        ORDER BY
            orderid
    )
FROM
    orderDetails
GROUP BY
    productID,
    orderID
LIMIT
    10

 * sqlite:///Northwind.db
Done.


OrderID,ProductID,Quantity,AVG(Quantity) OVER ( PARTITION BY productID ORDER BY orderid )
10285,1,45,45.0
10294,1,18,31.5
10317,1,20,27.666666666666668
10348,1,15,24.5
10354,1,12,22.0
10370,1,15,20.833333333333332
10406,1,10,19.285714285714285
10413,1,24,19.875
10477,1,15,19.333333333333332
10522,1,40,21.4


## Solutions

### Exercise 1

In [ ]:
%%sql

SELECT 
    OrderID, 
    OrderDate,
    RANK() OVER (ORDER BY OrderDate DESC) as Order_rank
FROM 
    Orders 
WHERE 
    CustomerID = 'ALFKI';

The `RANK()` window function is used here to rank each order of the customer with the ID `'ALFKI'` based on the `OrderDate`. The `DESC` keyword is used so that the most recent order gets the highest rank (i.e. 1).

### Exercise 2

In [23]:
%%sql

SELECT 
    OrderID, 
    Quantity, 
    SUM(Quantity) OVER (
    ORDER BY OrderID) as RunningTotal 
FROM 
    OrderDetails
GROUP BY 
    OrderID
limit 10;

 * sqlite:///Northwind.db
Done.


OrderID,Quantity,RunningTotal
10248,12,12
10249,9,21
10250,10,31
10251,6,37
10252,40,77
10253,20,97
10254,15,112
10255,20,132
10256,15,147
10257,25,172


The `SUM()` window function is used here to calculate a running total of the quantity of orders. The `ORDER BY` clause inside the `OVER()` clause ensures that the running total is calculated in the order of the `OrderID`. We then group our data by `OrderID`.

### Exercise 3

In [40]:
%%sql

SELECT 
    CustomerID, 
    OrderDate, 
    LAG(OrderDate, 1) OVER 
        (PARTITION BY CustomerID 
        ORDER BY OrderDate) as PrevOrderDate, 
    julianday(OrderDate)-
    julianday(LAG(OrderDate, 1) OVER
            (PARTITION BY CustomerID 
            ORDER BY OrderDate)) as DateDiff
FROM 
    Orders
limit 5

 * sqlite:///Northwind.db
Done.


CustomerID,OrderDate,PrevOrderDate,DateDiff
ALFKI,1997-08-25 00:00:00,None,None
ALFKI,1997-10-03 00:00:00,1997-08-25 00:00:00,39.0
ALFKI,1997-10-13 00:00:00,1997-10-03 00:00:00,10.0
ALFKI,1998-01-15 00:00:00,1997-10-13 00:00:00,94.0
ALFKI,1998-03-16 00:00:00,1998-01-15 00:00:00,60.0


The `LAG()` window function is used twice here, once to get the previous order date for each customer, and again to calculate the difference between the current order date and the previous order date. The `PARTITION BY` clause is used to separate the data into partitions based on the `CustomerID`. 

Since the `TIMESTAMPDIFF()` function in MySQL is not available in SQLite, we use the `julianday()` function to convert the dates to a floating point number, which we can then use for subtraction to find the difference between two dates.

### Exercise 4

In [52]:
%%sql

SELECT 
    OrderID, 
    ProductID, 
    Quantity,
    AVG(Quantity) OVER (PARTITION BY ProductID ORDER BY OrderID ROWS BETWEEN 2 PRECEDING AND 0 FOLLOWING) as MovingAvgQuantity
FROM 
    OrderDetails
ORDER BY 
    ProductID, 
    OrderID
LIMIT 10

 * sqlite:///Northwind.db
Done.


OrderID,ProductID,Quantity,MovingAvgQuantity
10285,1,45,45.0
10294,1,18,31.5
10317,1,20,27.666666666666668
10348,1,15,17.666666666666668
10354,1,12,15.666666666666666
10370,1,15,14.0
10406,1,10,12.333333333333334
10413,1,24,16.333333333333332
10477,1,15,16.333333333333332
10522,1,40,26.333333333333332


The `AVG()` window function is used here to calculate the moving average of `Quantity` for the last 3 orders (the current order and the two preceding orders) for each product. The window is defined using the `PARTITION BY` clause (to segment the data by `ProductID`) and the `ORDER BY` clause (to arrange the data in order of `OrderID`). The `ROWS BETWEEN` clause specifies the size and location of the window – in this case, **the current row and the two rows preceding it.**

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>